In [2]:
import numpy as np
import os
import tensorflow.compat.v1 as tf
import magenta.music as mm
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
#from google.colab import files

tf.disable_v2_behavior()

c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import argparse
from pythonosc import udp_client

parser = argparse.ArgumentParser()
# OSC server ip: '127.0.0.1'
#parser.add_argument("--ip", default='192.168.255.27', help="The ip of the OSC server")
parser.add_argument("--ip", default='192.168.193.27') # if in the same machine
parser.add_argument("--port", type=int, default=7400)

# Parse the arguments
args, unknown = parser.parse_known_args()

# Start the UDP Client
client = udp_client.SimpleUDPClient(args.ip, args.port)

In [4]:
# Parameters
BATCH_SIZE = 4
Z_SIZE = 512
TOTAL_STEPS = 512
BAR_SECONDS = 2.0
CHORD_DEPTH = 49

SAMPLE_RATE = 44100
SF2_PATH = '/content/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

In [5]:
# Functions

# Chord encoding tensor.
def chord_encoding(chord):
  index = mm.TriadChordOneHotEncoding().encode_event(chord)
  c = np.zeros([TOTAL_STEPS, CHORD_DEPTH])
  c[0,0] = 1.0
  c[1:,index] = 1.0
  return c

# Trim sequences to exactly one bar.
def trim_sequences(seqs, num_seconds=BAR_SECONDS):
  for i in range(len(seqs)):
    seqs[i] = mm.extract_subsequence(seqs[i], 0.0, num_seconds)
    seqs[i].total_time = num_seconds

# Consolidate instrument numbers by MIDI program.
def fix_instruments_for_concatenation(note_sequences):
  instruments = {}
  for i in range(len(note_sequences)):
    for note in note_sequences[i].notes:
      if not note.is_drum:
        if note.program not in instruments:
          if len(instruments) >= 8:
            instruments[note.program] = len(instruments) + 2
          else:
            instruments[note.program] = len(instruments) + 1
        note.instrument = instruments[note.program]
      else:
        note.instrument = 9

In [6]:
config = configs.CONFIG_MAP['hier-multiperf_vel_1bar_med_chords']
model = TrainedModel(
    config, 
    batch_size=BATCH_SIZE,
    checkpoint_dir_or_path='content/model_chords_fb64.ckpt')

INFO:tensorflow:Building MusicVAE model with HierarchicalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 512, 'z_size': 512, 'free_bits': 0.0, 'max_beta': 1.0, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [512, 512, 512], 'enc_rnn_size': [1024], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Hierarchical Encoder:
  input length: 512
  level lengths: [64, 8]

INFO:tensorflow:Level 0 splits: 8
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [1024]

INFO:tensorflow:Level 1 splits: 1
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [1024]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 512
  level output lengths: [8, 64]

INFO:tensorflow:
Decoder Cells:
  units: [512, 512, 512]



c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\magenta\models\music_vae\lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\magenta\contrib\rnn.py:750: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\magenta\contrib\rnn.py:753: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\magenta\models\music_vae\base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\magenta-env\lib\site-packages\magenta\models\music_vae\base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from content/model_chords_fb64.ckpt


In [7]:
double_seq = []
chord_1 = 'C' #@param {type:"string"}
#chord_2 = 'Caug' #@param {type:"string"}
#chord_3 = 'Am' #@param {type:"string"}
#chord_4 = 'E' #@param {type:"string"}
chords = [chord_1]

temperature = 0.2 #@param {type:"slider", min:0.01, max:1.5, step:0.01}
z = np.random.normal(size=[1, Z_SIZE])
seqs = [
    model.decode(length=TOTAL_STEPS, z=z, temperature=temperature,
                 c_input=chord_encoding(c))[0]
    for c in chords
]


In [8]:
double_seq = seqs + seqs
trim_sequences(double_seq)
fix_instruments_for_concatenation(double_seq)
prog_ns = concatenate_sequences(double_seq)

#play(prog_ns)
mm.plot_sequence(prog_ns)

Loading BokehJS ...

In [14]:
for notes in prog_ns.notes:
    osc_message = "instrument: " + str(notes.instrument) + " pitch: " + str(notes.pitch) + " velocity: " + str(notes.velocity) + " start_time: " + str(notes.start_time) + " end_time: " + str(notes.end_time)
    client.send_message("note", osc_message)
    print(osc_message)

['instrument:', '1', 'pitch:', '60', 'velocity:', '65', 'start_time:', '0.0', 'end_time:', '2.0']
['instrument:', '1', 'pitch:', '64', 'velocity:', '65', 'start_time:', '0.0', 'end_time:', '2.0']
['instrument:', '1', 'pitch:', '67', 'velocity:', '65', 'start_time:', '0.0', 'end_time:', '2.0']
['instrument:', '2', 'pitch:', '48', 'velocity:', '113', 'start_time:', '0.0', 'end_time:', '0.16666666666666666']
['instrument:', '3', 'pitch:', '72', 'velocity:', '97', 'start_time:', '0.0', 'end_time:', '0.25']
['instrument:', '9', 'pitch:', '36', 'velocity:', '97', 'start_time:', '0.0', 'end_time:', '0.3333333333333333']
['instrument:', '9', 'pitch:', '49', 'velocity:', '97', 'start_time:', '0.0', 'end_time:', '0.3333333333333333']
['instrument:', '2', 'pitch:', '48', 'velocity:', '97', 'start_time:', '0.25', 'end_time:', '0.41666666666666663']
['instrument:', '3', 'pitch:', '72', 'velocity:', '97', 'start_time:', '0.375', 'end_time:', '0.7083333333333333']
['instrument:', '2', 'pitch:', '48',

In [49]:
while True:
    
    

SyntaxError: unexpected EOF while parsing (851964165.py, line 2)

In [ ]:
from pythonosc.dispatcher import Dispatcher
from pythonosc import osc_server
dispatcher = Dispatcher()
server = osc_server.ThreadingOSCUDPServer((args.ip, args.port), dispatcher)
server.serve_forever()
